In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# --- Hyperparams ---
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 300  
learning_rate = 1e-2
device = torch.device('mps') or ('cuda' if torch.cuda.is_available() else 'cpu')
eval_iters = 200
n_embd = 32
print(device)
# -----------------

torch.manual_seed(1337)

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)

# "Tokenizer". Simple; pros: small vocab size, cons: long sequences
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: [itos[i] for i in l]
encoded = encode("HALLO")
encoded, decode(encoded)

torch.manual_seed(1337)

data = torch.tensor(encode(text), dtype=torch.long)
n = int(.9*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split="train"):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y, = x.to(device), y.to(device)
  return x, y

@torch.no_grad() 
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Bigram(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_embd = self.token_embedding_table(idx) # (B, T, C)
    pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = tok_embd + pos_embd
    logits = self.lm_head(x)
    # print(logits.shape)
    # Reshape for corssentropy

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C )
      targets = targets.view (B*T)
      # print(logits.shape, targets.shape)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

model = Bigram()
model = model.to(device)

optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iters):
    xb, yb = get_batch()

    # Evaluate Loss
    logits, loss = model(xb, yb)
    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

# "".join([decode(batch) for batch in m.generate(torch.zeros((1, 1), dtype=torch.long), 100).tolist()][0])
context = torch.zeros((2, 1, 1), dtype=torch.long, device=device)
print(''.join(decode(model.generate(context, 500)[0].tolist())))


mps
step 0: train loss 4.4800, val loss 4.4803
step 300: train loss 2.5399, val loss 2.5577
step 600: train loss 2.5167, val loss 2.5340
step 900: train loss 2.4968, val loss 2.5147

Foasth pr!
SKixchaPENGRYIOLOMUKEE&exby:
QUS:
3 COLineg agntheprdrrknteckeyr PHENThe?
TyONGrsothy.
DLOPELABUSimppry PS:
MANIF
Shbur$erwixikns
Fokncaknd-htNRLYoun, KGLIOMOMu gwirexthe.
AUFOMETHAnthcus;
NDUFr
TEKYCONTRDURDUKIEUjerks?
STh!
Tus
LUSA'st BEFoghy whe f,
SThogiznk&ofachang!
&CETDWler'dsuqughold ark'dz

PESThe
TJUCENCTINGBundmzeyby hindyongmy!
HENRRVof,
CI'Thy.
MARBRUCUMy,---GHIOfourknghm goutheppmplkedextherex'digackefithrkn'GENUS:
Y laghindKINGRBedouerdjurd.
JKEMjurKENGHEOKIFe.
IINGS:
Y
